### Copyright-protected material, all rights reserved. (c) University of Vienna.
_Copyright Notice of the corresponding course at Moodle applies. <br> Only to be used in the MRE course._

# MRE Assignment 1 - Digital Image Processing

In this assignment you will use Python (using Pillow or OpenCV) to load, transcode and store images. You will also use the libraries to extract some basic metadata from the images and store them in a data structure for easy access.

In this notebook, you will implement your solution. This notebook will be imported into the "*_def.ipynb" notebook.

Of course you can include code for testing your implementation in this implementation notebook, but code for testing and output generated for testing is not going to be assessed.

Of course, your code for the solutions in this notebook will be inspected and is subject to grading.


## Setup

For general installation instructions, please refer to the ressources given for all the assignments [in Moodle](https://moodle.univie.ac.at/course/view.php?id=260637#section-13).

If the cell below executes without error, you can start the assignment!

In [16]:
# -------- Imports --------
# Please do not change the contents of this cell!

# In case you work in a local environment on your own machine,
# how to install the required packages:
#   PIP:     pip install <name> / pip install --upgrade <name>
#   CONDA:   conda install -c conda-forge <name>  /  conda install -c anaconda <name>


# Imports required by us.
import cv2                             # opencv-python
from PIL import Image                  # pillow
from PIL.ExifTags import TAGS
from matplotlib import pyplot as plt   # matplotlib
from sklearn.metrics.pairwise import cosine_similarity  # scikit-learn
import numpy as np                     # numpy
import pandas as pd                    # pandas
from IPython.display import display    # packaged with python

# This directive will allow matplotlib to render interactive plots in the notebook.
%matplotlib widget

In the cells below, place your own imports, global variables, (helper) functions and classes. Feel free to add cells here as you see fit.

In [17]:
# Please place your own imports here.
import os
from PIL import ImageFilter
from PIL import ImageDraw

In [18]:
# Place any helper functions, global variables and classes here.

## Task 1.1: Image formats transformation and adding filters


In [19]:
# Write your function here.

# Transcodes images to JPEG format of adjustable quality (for JPEG).
def JPEGImageConverter(inputImg: str, outputDir: str, quality: float = 1.0) -> None:
    os.makedirs(outputDir, exist_ok=True)   # make output directory if not existant
    
    if os.path.isdir(inputImg):
        for filename in os.listdir(inputImg):
            filepath = os.path.join(inputImg, filename)
            
            fileName, fileExtension = os.path.splitext(filename)
            if os.path.isfile(filepath) and fileExtension.upper() in ['.JPG', '.JPEG', '.BMP', '.PNG', '.GIF']: #just take these extensions
                convertFile(filepath, outputDir, quality)
    else:
        convertFile(inputImg, outputDir, quality)

def convertFile(inputImg: str, outputDir: str, quality: float = 1.0):
    with Image.open(inputImg) as image:
        originalName = os.path.basename(inputImg)
        fileName, fileExtension = os.path.splitext(originalName)
        
        if quality == 1.0:
            outputName = f"{originalName}.jpg"
        else:
            outputName = f"{originalName}-{quality}.jpg" 
            
        outputPath = os.path.join(outputDir, outputName)
                
        image.convert("RGB").save(outputPath, format="JPEG", quality=int(quality * 100))

    

In [20]:
# Test your function here.
#JPEGImageConverter("images/jawa-1.jpg", "./output_dir", 0.77)

In [21]:
# Add Blur filter
def BlurImage(inputImg: str) -> None:
    with Image.open(inputImg) as image:
        originalName = os.path.basename(inputImg)
        fileName, fileExtension = os.path.splitext(originalName)
        outputName = f"{originalName}-blurred.jpg"
        outputPath = os.path.join("./output_dir", outputName)
        
        blurred = image.filter(filter=ImageFilter.BLUR)
        blurred.convert("RGB").save(outputPath, format="JPEG", quality=int(100))

In [22]:
# Test your function here.
#BlurImage("images/zebra3m.jpg")

## Task 1.2:  Extract / Get Image Metadata

In [23]:
# Write your function here. 

# Extracts metadata from an image and places them into a Pandas DataFrame.
def ImageMetadataGenerator(inputImage: str) -> pd.DataFrame:
    df = pd.DataFrame(columns=['File', 'Width', 'Height', 'Components', 'Channels', 'BitsPerPixel', 'ColorSpaceType', 'Format', 'Pixels', 'HasAlphaChannel', 'SizeWithoutCompressionInKB'])
    
    listOfImagePaths = []
    
    if os.path.isdir(inputImage):
        for filename in os.listdir(inputImage):
            filepath = os.path.join(inputImage, filename)
            fileName, fileExtension = os.path.splitext(filename)
            if os.path.isfile(filepath) and fileExtension.upper() in ['.JPG', '.JPEG', '.BMP', '.PNG', '.GIF']: #just take these extensions
                listOfImagePaths.append(filepath)
    else:
        listOfImagePaths.append(inputImage)
    
    for imagePath in listOfImagePaths:
        print(imagePath) # Weg damit
        with Image.open(imagePath) as image:
            hasAlphaChannel = False
            
            file = image.filename
            width = image.width
            height = image.height
            
            bands = image.getbands()
            numChannels = len(bands)
            numComponents = len(bands)
            if 'A' in bands:
                numComponents -= 1
                hasAlphaChannel = True
            
            bitsPerChannel = 8 #image.getchannel(0).bits # TODOOOOOOOOOOOOOO
            bitsPerPixel = bitsPerChannel * numChannels
            
            colorSpaceType = image.mode
            
            imageFormat = image.format
            pixels = width*height
            sizeWithoutCompressionInKB = ((pixels*bitsPerPixel)/8)/1024 # theoretical size without compression
            
            df = df.append({
                    'File': file,
                    'Width': width,
                    'Height': height,
                    'Components': numComponents,
                    'Channels': numChannels,
                    'BitsPerPixel': bitsPerPixel,
                    'ColorSpaceType': colorSpaceType,
                    'Format': imageFormat,
                    'Pixels': pixels,
                    'HasAlphaChannel': hasAlphaChannel,
                    'SizeWithoutCompressionInKB': sizeWithoutCompressionInKB
                }, ignore_index=True)
        
    return df

In [24]:
# Test your function here.
display(ImageMetadataGenerator("images/"))


with Image.open("images/b17barb026_dithered.gif") as image:
    #print(image.getextrema()[1])
    pass

images/b17barb026_dithered.gif
images/ml-3.jpg
images/ml-1.jpg
images/zebra3m.JPG
images/onb-2.jpg
images/b17maartent1427.gif
images/picture_of_coati_dithered.png
images/onb-3.jpg
images/ml-5.jpg
images/b2seaanimals005.jpg
images/onb-1.jpg
images/jawa-2.2.jpg
images/peafowl_picture1.JPG
images/b3_seaanimals014.jpg
images/par-1.jpg
images/jawa-2.1.jpg
images/ml_2.jpg
images/blue_sky4m.JPG
images/blue_sky5m.JPG
images/vie-1.jpg
images/b20nature_plants245.jpg
images/vie-3.jpg
images/vie-2.jpg
images/b1murene.jpg
images/the_real_firefox.png
images/ca-1.jpg
images/m-1.jpg
images/m-3.jpg
images/picture_of_coati.JPG
images/underwater_rugby.JPG
images/m-2.jpg
images/b9nature_plants064.jpg
images/b17maartent1427.jpg
images/lib-1.jpg
images/the_real_firefox_dithered.bmp
images/b1shell008.jpg
images/ca-21.jpg
images/lib-2.jpg
images/picture_of_goat11.JPG
images/m-4.jpg
images/jawa-4.jpg
images/elephant14m.JPG
images/m-4.1.jpg
images/elephant14m.png
images/lib1-xMedUni_Wien_Josephinum_Bibliothek-m

/var/folders/kv/rlm1b_915wsbvd995jlr3f880000gn/T/ipykernel_2737/887948427.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/var/folders/kv/rlm1b_915wsbvd995jlr3f880000gn/T/ipykernel_2737/887948427.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/var/folders/kv/rlm1b_915wsbvd995jlr3f880000gn/T/ipykernel_2737/887948427.py:43: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = df.append({
/var/folders/kv/rlm1b_915wsbvd995jlr3f880000gn/T/ipykernel_2737/887948427.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
/var/folders/kv/rlm1b_915wsb

,File,Width,Height,Components,Channels,BitsPerPixel,ColorSpaceType,Format,Pixels,HasAlphaChannel,SizeWithoutCompressionInKB
0,images/b17barb026_dithered.gif,1024,768,1,1,8,P,GIF,786432,False,768.000000
1,images/ml-3.jpg,600,400,3,3,24,RGB,JPEG,240000,False,703.125000
2,images/ml-1.jpg,620,342,3,3,24,RGB,JPEG,212040,False,621.210938
3,images/zebra3m.JPG,800,532,3,3,24,RGB,JPEG,425600,False,1246.875000
4,images/onb-2.jpg,348,348,3,3,24,RGB,JPEG,121104,False,354.796875
5,images/b17maartent1427.gif,50,38,1,1,8,P,GIF,1900,False,1.855469
6,images/picture_of_coati_dithered.png,1023,558,1,1,8,P,PNG,570834,False,557.455078
7,images/onb-3.jpg,480,480,3,3,24,RGB,JPEG,230400,False,675.000000
8,images/ml-5.jpg,600,450,3,3,24,RGB,MPO,270000,False,791.015625
9,images/b2seaanimals005.jpg,1600,1200,3,3,24,RGB,JPEG,1920000,False,5625.000000


## Task 1.3: Drawing circles around objects in an image

In [188]:
# Write your function here. 

# Identifies shapes in an images and draws circles around them.
def IdentifyObjects(inputImage: str) -> None:
    image = cv2.imread(inputImage)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 0)
    edges = cv2.Canny(blurred, 50, 150)
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        if cv2.contourArea(contour) > 120:
            (x, y), radius = cv2.minEnclosingCircle(contour)
            center = (int(x), int(y))
            radius = int(radius)
            cv2.circle(image, center, radius, (0, 0, 255), 2)
    
    cv2.imshow("Identified objects", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [189]:
# Test your function here.
#IdentifyObjects("images/Task1.3/partyballoons-1.jpg")
#IdentifyObjects("images/Task1.3/partyballoons-2.jpg")

## Task 1.4: Duplicate image finder using cosine similarity

In [ ]:
# Calculates a histogram for one image.
def ImageHistogramGenerator(inputImage: str) -> np.array:
    pass

In [ ]:
# Test your function here.

In [ ]:
# Find duplicate images and return their similarity scores in a dataframe.
def FindDuplicateImages(inputDir1 : str, inputDir2: str, similarityThreshold: float = 1.0) -> pd.DataFrame:
    pass

In [ ]:
# Test your function here.